## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. They often do that by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will look at how you can extract BGP configuration and session status information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

/Users/samir/git/pybatfish/pybatfish/client/commands.py:48: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### View BGP Configuration for ALL devices and ALL VRFs
Batfish makes BGP Configuration settings in the network easily accessible. Let's take a look at how you can retrieve the specific information you want.

In [3]:
# Get routing tables for all nodes and VRFs
bgp_config = bfq.bgpProperties().answer().frame()
bgp_config

,Node,VRF,Router_ID,Multipath_EBGP,Multipath_IBGP,Tie_Breaker,Passive_Neighbors,Multipath_Match_Mode
0,as3core1,default,3.10.1.1,True,True,ARRIVAL_ORDER,"[3.1.1.1/32, 3.2.2.2/32]",EXACT_PATH
1,as2border2,default,2.1.1.2,True,True,ARRIVAL_ORDER,"[2.1.2.1/32, 2.1.2.2/32, 10.23.21.3/32]",EXACT_PATH
2,as2border1,default,2.1.1.1,True,True,ARRIVAL_ORDER,"[2.1.2.1/32, 2.1.2.2/32, 10.12.11.1/32]",EXACT_PATH
3,as3border2,default,3.2.2.2,True,True,ARRIVAL_ORDER,"[3.10.1.1/32, 10.13.22.1/32]",EXACT_PATH
4,as2dist1,default,2.1.3.1,True,True,ARRIVAL_ORDER,"[2.1.2.1/32, 2.1.2.2/32, 2.34.101.4/32]",EXACT_PATH
5,as1border2,default,1.2.2.2,True,True,ARRIVAL_ORDER,"[1.10.1.1/32, 10.13.22.3/32, 10.14.22.4/32]",EXACT_PATH
6,as3border1,default,3.1.1.1,True,True,ARRIVAL_ORDER,"[3.10.1.1/32, 10.23.21.2/32]",EXACT_PATH
7,as2core1,default,2.1.2.1,True,True,ARRIVAL_ORDER,"[2.1.1.1/32, 2.1.1.2/32, 2.1.3.1/32, 2.1.3.2/32]",EXACT_PATH
8,as1border1,default,1.1.1.1,True,True,ARRIVAL_ORDER,"[1.10.1.1/32, 3.2.2.2/32, 5.6.7.8/32, 10.12.11.2/32]",EXACT_PATH
9,as2core2,default,2.1.2.2,True,True,ARRIVAL_ORDER,"[2.1.1.1/32, 2.1.1.2/32, 2.1.3.1/32, 2.1.3.2/32]",EXACT_PATH


### View BGP Session Status

In [4]:
?bfq.bgpSessionStatus

In [5]:
# Get the routing table for the 'default' VRF on border routers of as1
# using BF parameters

bgp_sessions = bfq.bgpSessionStatus(includeEstablishedCount=True).answer().frame()
bgp_sessions

,Node,Local_IP,VRF,Local_Interface,Remote_Node,Remote_Prefix,Session_Type,Configured_Status,Established_Neighbors
0,as2border1,10.12.11.2,default,as2border1:GigabitEthernet0/0,as1border1,10.12.11.1/32,EBGP_SINGLEHOP,UNIQUE_MATCH,1
1,as3core1,3.10.1.1,default,as3core1:Loopback0,as3border2,3.2.2.2/32,IBGP,UNIQUE_MATCH,1
2,as1border2,1.2.2.2,default,as1border2:Loopback0,as1core1,1.10.1.1/32,IBGP,UNIQUE_MATCH,1
3,as1border1,None,default,None,None,3.2.2.2/32,EBGP_SINGLEHOP,NO_LOCAL_IP,-1
4,as2core1,2.1.2.1,default,as2core1:Loopback0,as2dist2,2.1.3.2/32,IBGP,UNIQUE_MATCH,1
5,as3border2,10.13.22.3,default,as3border2:GigabitEthernet0/0,as1border2,10.13.22.1/32,EBGP_SINGLEHOP,UNIQUE_MATCH,1
6,as2core2,2.1.2.2,default,as2core2:Loopback0,as2dist2,2.1.3.2/32,IBGP,UNIQUE_MATCH,1
7,as1border1,1.1.1.1,default,as1border1:Loopback0,as1core1,1.10.1.1/32,IBGP,UNIQUE_MATCH,1
8,as1core1,1.10.1.1,default,as1core1:Loopback0,as1border1,1.1.1.1/32,IBGP,UNIQUE_MATCH,1
9,as1border1,None,default,None,None,5.6.7.8/32,EBGP_SINGLEHOP,NO_LOCAL_IP,-1


With Batfish you can easily retrieve information about BGP configuration and session status information.

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics like path analysis, debugging ACLs and firewall rules, validating routing policy, etc.. so stay tuned! 

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.